In [2]:
import pandas as pd

In [3]:
import json
from pandas.io.json import json_normalize

In [5]:
world_bank_projects_df = pd.read_json('data/world_bank_projects.json')

In [6]:
df = world_bank_projects_df
len(df[df.country_namecode.isnull()])

0

In [5]:
# Find the 10 countries with most projects
# Countries can be identified by either countryname, countrycode or country_namecode
# Neither of these columns contain a NaN value
# Example: df = world_bank_projects_df
#          len(df[df.country_namecode.isnull()])
# returns 0
# Countryname i most appropriate as it reflects the country name in a way that seems understandable for anyone
# It might by a good idea though to include at least the countrycode in case there is a need to merge with country related data
# in any future/oncoming requirements
df.groupby(['countryname','countrycode','country_namecode']).size().nlargest(10)
# QUESTION: How to add headers/column names?
#print ("The 10 countries with most projects")
#df.countryname.value_counts().head(10)

countryname                      countrycode  country_namecode                    
People's Republic of China       CN           People's Republic of China!$!CN         19
Republic of Indonesia            ID           Republic of Indonesia!$!ID              19
Socialist Republic of Vietnam    VN           Socialist Republic of Vietnam!$!VN      17
Republic of India                IN           Republic of India!$!IN                  16
Republic of Yemen                RY           Republic of Yemen!$!RY                  13
Kingdom of Morocco               MA           Kingdom of Morocco!$!MA                 12
Nepal                            NP           Nepal!$!NP                              12
People's Republic of Bangladesh  BD           People's Republic of Bangladesh!$!BD    12
Africa                           3A           Africa!$!3A                             11
Republic of Mozambique           MZ           Republic of Mozambique!$!MZ             11
dtype: int64

In [12]:
# Find the top 10 major project themes (using column 'mjtheme_namecode')
# First I collected the values in the column 'mjtheme_namecode' by running df and exporting the result to a csv file:
#    df.to_csv('World banl projects.csv', encoding = 'utf-8')
# which I reviewed in Excel
# Another way is to capture mjtheme_namecode in a series or list and retrieve some values:
#    data = df['mjtheme_namecode'] -> creates a data frame
#    data = df['mjtheme_namecode'].values.tolist() -> creates a list
#    retrieve value for index 100 -> json_normalize(data[11])
# Column 'mjtheme_namecode' contains nested groups, consisting of code and name
# It's apparent that we can normalize the data using json_normalize
print('Top 10 major project themes')
data = json.load((open('data/world_bank_projects.json')))
project_themes = json_normalize(data, 'mjtheme_namecode')
project_themes.name.value_counts().head(10)
#alternative: project_themes.groupby(['name']).size().nlargest(10)

Top 10 major project themes


Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

In [13]:
# Create a dataframe with the missing names filled in
# First, check how many names are missing, use this as a "zero reference point":
#     project_themes[project_themes['name'].str.len() ==0]
# To count the number of missing names you can use:
#     project_themes[project_themes['name'].str.len() ==0].count()
#Next, create a dataframe and fill in the missing name values
#lookup = project_themes[project_themes['name'].str.len() !=0].groupby(['name','code']).size() -> creates a Series
#lookup = project_themes[project_themes['name'].str.len() !=0].groupby(['code','name']).count()-> creates an empty DataFrame,
# lookup.columns.tolist() results in []
lookup = project_themes[project_themes['name'].str.len() !=0].groupby(['name','code']).size().reset_index()
# REMARK, apparently you have to use reset_index. Why, what does it do exactly?
lookup.columns.tolist()
# We don't want/need the last column hence delete it
del lookup[0]
#merge the dataframe containing all codes/names Lookup) with the dataframe with missing names:
result = pd.merge(lookup, project_themes,how='left', left_on='code', right_on='code', suffixes=['','_lookup'])
#drop the column you added a suffix to:
del result['name_lookup']
#check for any missing names in result:
result[result['name'].str.len() ==0].count()
#Select the top 10 major project themes:
print('Top 10 major project themes (final)')
result.groupby(['name']).size().nlargest(10)